In [1]:
#1e.
# The workhorses
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.special

# Plotting modules
import matplotlib.pyplot as plt

# This is to enable inline displays for the purposes of the tutorial
%matplotlib inline

# This enables SVG graphics inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Seaborn makes plots look nice
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.linewidth': 2.5})
sns.set_style('darkgrid', {'axes.facecolor': '(0.875, 0.875, 0.9)'})



In [64]:
df= pd.read_csv("data/reeves_et_al/reeves_dv_profile_over_time.csv", comment="#")

#each column in original dataset represents ListOfNuclearCycles
ListOfNuclearCycles=["14","14","13","13","12","12","11","11"]

VarNames = ["DVCoordinate","Intensity"]*4

In [66]:
df.columns=pd.MultiIndex.from_arrays((ListOfNuclearCycles, VarNames), 
                                        names=['NC', 'variable'])

In [69]:
#getting a tidy dataset. the dataframe was not big, hence we just do it manually

TidyData=pd.concat((df["14"],df["13"],df["12"],df["11"])).reset_index(drop=True)
ListOfNC=sum(list(([int("14")]*44,[int("13")]*44,[int("12")]*44,[int("11")]*44)),[])
TidyData["NC"]=ListOfNC
TidyData = TidyData.dropna().reset_index(drop=True)

In [70]:
TidyData[TidyData["Intensity"]<2*np.min(TidyData["Intensity"])]

variable,DVCoordinate,Intensity,NC
0,0.7813,1.6512,14
1,0.8565,1.4200,14
2,0.8986,1.3845,14
3,0.9286,1.3960,14
4,-0.9356,1.1741,14
5,-0.8846,1.3896,14
6,-0.7627,0.9789,14
7,-0.7340,0.9505,14
8,-0.7073,1.0104,14
9,-0.6830,1.2662,14
